# **CA4. Multiprocessing**
**Sansores Cruz Angel David\
Data Engineering\
Universidad Politécnica de Yucatán\
Ucú, Yucatán, México\
2109139@upy.edu.mx** \

# Part 1. Benchmarking and Profiling

**The Julia set is an interesting CPU-bound problem for us to begin with. It is a fractal sequence that generates a complex output image, named after Gaston Julia.**

1. Read the sections “Introducing the Julia Set” and “Calculating the Full Julia Set” on Chapter 2. Profiling to Find Bottlenecks from the book: **M. Gorelick & I. Ozsvald(2020). High Performance Python. Practical Performant Programming for Humans. Second Edition. United States of America: O’Reilly Media, Inc.** Implement the chapter functions (Example 2-1, 2-2, 2-3 and 2-4) on Python in order to calculate the Julia Set. Make the representation for the false gray and pure gray scale.

In [4]:
import time

In [5]:
#Example 2-1. Defining global constants for the coordinate space
"""Julia set generator without optional PIL-based image drawing"""
# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

In [6]:
# Example 2-2. stablishing the coordinate lists as inputs to our calculation function
def calc_pure_python(desired_width, max_iterations):
  """Create a list of complex coordinates (zs) and complex parameters (cs),
 build Julia set"""
  x_step = (x2 - x1) / desired_width
  y_step = (y1 - y2) / desired_width
  x = []
  y = []
  ycoord = y2
  while ycoord > y1:
    y.append(ycoord)
    ycoord += y_step
  xcoord = x1
  while xcoord < x2:
    x.append(xcoord)
    xcoord += x_step
# build a list of coordinates and the initial condition for each cell.
# Note that our initial condition is a constant and could easily be removed,
# we use it to simulate a real-world scenario with several inputs to our
# function
  zs = []
  cs = []
  for ycoord in y:
    for xcoord in x:
      zs.append(complex(xcoord, ycoord))
      cs.append(complex(c_real, c_imag))
  print("Length of x:", len(x))
  print("Total elements:", len(zs))
  start_time = time.time()
  output = calculate_z_serial_purepython(max_iterations, zs, cs)
  end_time = time.time()
  secs = end_time - start_time
  print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")
# This sum is expected for a 1000^2 grid with 300 iterations
# It ensures that our code evolves exactly as we'd intended
  assert sum(output) == 33219980

In [7]:
#Example 2-3. Our CPU-bound calculation function
def calculate_z_serial_purepython(maxiter, zs, cs):
  """Calculate output list using Julia update rule"""
  output = [0] * len(zs)
  for i in range(len(zs)):
    n = 0
    z = zs[i]
    c = cs[i]
    while abs(z) < 2 and n < maxiter:
      z = z * z + c
      n += 1
    output[i] = n
  return output

**Once we run the next code, we see some output about the complexity of the problem:**\
**running the above produces:**\
Length of x: 1000\
Total elements: 1000000\
calculate_z_serial_purepython took 8.087012767791748 seconds

In [8]:
if __name__ == "__main__":
# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
  calc_pure_python(desired_width=1000, max_iterations=300)


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 5.4955644607543945 seconds


# Representation for **False Gray** and **Pure Grayscale**

In [9]:
%pip install Pillow
from PIL import Image
import numpy as np
import array


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


2. Define a new function, timefn, which takes a function as an argument: the inner function, measure_time, takes *args (a variable number of positional arguments) and **kwargs (a variable number of key/value arguments) and passes them through to fn for execution. Decorate calculate_z_serial_purepython with @timefn to profile it. Implement Example 2-5 and adapt your current source code.

In [10]:
# Example 2-5. Defining a decorator to automate timing measurements
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time

In [11]:
@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    '''Calculate output list using Julia update rule'''
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

In [12]:
if __name__ == "__main__":
    calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 6.549245357513428 seconds
calculate_z_serial_purepython took 6.549245357513428 seconds


3. Use the timeit modeule to get a coarse measurement of the execution speed of the CPU-bound function. Runs 10 loops with 5 repetitions. Show how to do the
measurement on the command line and on a Jupyter Notebook.

In [13]:
%timeit -r 5 -n 10 calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 6.025115489959717 seconds
calculate_z_serial_purepython took 6.025115489959717 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.883240461349487 seconds
calculate_z_serial_purepython took 8.883240461349487 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.317840337753296 seconds
calculate_z_serial_purepython took 8.317840337753296 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.40365743637085 seconds
calculate_z_serial_purepython took 8.40365743637085 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.699599027633667 seconds
calculate_z_serial_purepython took 8.699599027633667 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.682618379592896 seconds
calculate_z_serial_purepy

4. Use the cProfile module to profile the source code (.py). Sort the results by the time spent inside each function. This will give a view into the slowest parts. Analyze the output and make a syntesis of the findings. Show how to use the cProfile module on the command line and on a Jupyter Notebook.

In [14]:
import cProfile
import time

# Attempt to import the profile decorator from line_profiler if installed
try:
    from line_profiler import profile
except ImportError:
    def profile(func):
        """ Dummy decorator in case line_profiler is not installed. """
        return func

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

@profile
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

@profile
def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y2 - y1) / desired_width
    x = [x1 + i * x_step for i in range(desired_width)]
    y = [y1 + i * y_step for i in range(desired_width)]
    zs = [complex(x[i % desired_width], y[i // desired_width]) for i in range(desired_width ** 2)]
    cs = [complex(c_real, c_imag) for i in range(desired_width ** 2)]

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for a 1000^2 grid with 300 iterations

if __name__ == "__main__":
    cProfile.run('calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)', 'profile_stats')


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 15.080909490585327 seconds


In [15]:
import pstats
p = pstats.Stats('profile_stats')
p.sort_stats('cumulative').print_stats(10)  # Adjust this number to display more or fewer lines

Thu Apr  4 20:14:08 2024    profile_stats

         34220127 function calls in 15.806 seconds

   Ordered by: cumulative time
   List reduced from 29 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.806   15.806 {built-in method builtins.exec}
        1    0.026    0.026   15.806   15.806 <string>:1(<module>)
        1    0.000    0.000   15.780   15.780 C:\Users\angel\AppData\Local\Temp\ipykernel_17672\2613920211.py:30(calc_pure_python)
        1   10.286   10.286   15.080   15.080 C:\Users\angel\AppData\Local\Temp\ipykernel_17672\2613920211.py:16(calculate_z_serial_purepython)
 34219980    4.794    0.000    4.794    0.000 {built-in method builtins.abs}
        1    0.447    0.447    0.447    0.447 C:\Users\angel\AppData\Local\Temp\ipykernel_17672\2613920211.py:37(<listcomp>)
        1    0.241    0.241    0.241    0.241 C:\Users\angel\AppData\Local\Temp\ipykernel_17672\2613920211.py:38(<listcomp>)


5. Use snakeviz to get a high-level understanding of the cPrifile statistics file. Analyze the output and make a syntesis of the findings.

In [16]:
!pip install snakeviz


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
if __name__ == "__main__":
    cProfile.run('calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)', 'profile_stats')

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 14.296393394470215 seconds


In [18]:
!snakeviz 'profile_stats'

snakeviz: error: the path c:\Users\angel\Documents\7mo\CA3.-High-Performance-Python\Part 1_Benchmarking and Profiling\'profile_stats' does not exist

usage: snakeviz [-h] [-v] [-H ADDR] [-p PORT] [-b BROWSER_PATH] [-s] filename

Start SnakeViz to view a Python profile.

positional arguments:
  filename              Python profile to view

options:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -H ADDR, --hostname ADDR
                        hostname to bind to (default: 127.0.0.1)
  -p PORT, --port PORT  port to bind to; if this port is already in use a free
                        port will be selected automatically (default: 8080)
  -b BROWSER_PATH, --browser BROWSER_PATH
                        name of webbrowser to launch as described in the
                        documentation of Python's webbrowser module:
                        https://docs.python.org/3/library/webbrowser.html
  -s, --server          st

6. Use the line_profiler and kernprof file to profile line-by-line the function
calculate_z_serial_purepython. Analyze the output and make a syntesis of the
findings.

In [19]:
!pip install line_profiler


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
%load_ext line_profiler

In [21]:
import time

# Attempt to import the profile decorator from line_profiler if installed
try:
    from line_profiler import profile
except ImportError:
    def profile(func):
        """ Dummy decorator in case line_profiler is not installed. """
        return func

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

@profile
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

@profile
def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y2 - y1) / desired_width
    x = [x1 + i * x_step for i in range(desired_width)]
    y = [y1 + i * y_step for i in range(desired_width)]
    zs = [complex(x[i % desired_width], y[i // desired_width]) for i in range(desired_width ** 2)]
    cs = [complex(c_real, c_imag) for i in range(desired_width ** 2)]

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for a 1000^2 grid with 300 iterations

# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 10.298967599868774 seconds


In [22]:
%lprun -f calculate_z_serial_purepython calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 104.47628998756409 seconds


Timer unit: 1e-07 s

Total time: 67.1853 s
File: C:\Users\angel\AppData\Local\Temp\ipykernel_17672\2939688537.py
Function: calculate_z_serial_purepython at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                           @profile
    16                                           def calculate_z_serial_purepython(maxiter, zs, cs):
    17                                               """Calculate output list using Julia update rule"""
    18         1      72355.0  72355.0      0.0      output = [0] * len(zs)
    19   1000001    5224260.0      5.2      0.8      for i in range(len(zs)):
    20   1000000    4645870.0      4.6      0.7          n = 0
    21   1000000    5667042.0      5.7      0.8          z = zs[i]
    22   1000000    5161151.0      5.2      0.8          c = cs[i]
    23  34219980  274462572.0      8.0     40.9          while abs(z) < 2 and n < maxiter:
    24  33219980  201096046.0      6.1     29.9              z = z

7. Use the memory_profiler to diagnose memory usage. Analyze the output and make
a syntesis of the findings.

In [23]:
!pip install memory_profiler
!pip install psutil


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\angel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
%load_ext memory_profiler

In [25]:
%%memit
# Import required libraries
import time

# Define the boundaries of the complex space and specific parameters
lower_x, upper_x, lower_y, upper_y = -1.8, 1.8, -1.8, 1.8
real_part, imaginary_part = -0.62772, -0.42193

def calculate_z_serial(max_iterations, zs, cs):
    """Calculates and returns a list with the iteration count for each point."""
    result = [0] * len(zs)
    for i in range(len(zs)):
        iterations = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and iterations < max_iterations:
            z = z**2 + c
            iterations += 1
        result[i] = iterations
    return result

def generate_julia(draw_output, desired_width, max_iterations):
    """Generates and computes the Julia set for a set of complex coordinates."""
    x_step = (upper_x - lower_x) / desired_width
    y_step = (upper_y - lower_y) / desired_width
    x_values = [lower_x + i * x_step for i in range(desired_width)]
    y_values = [lower_y + i * y_step for i in range(desired_width)]
    zs = [complex(x_values[i % desired_width], y_values[i // desired_width]) for i in range(desired_width ** 2)]
    cs = [complex(real_part, imaginary_part) for i in range(desired_width ** 2)]

    print("Length of x values:", len(x_values))
    print("Total elements:", len(zs))
    start_time = time.time()
    result = calculate_z_serial(max_iterations, zs, cs)
    end_time = time.time()
    execution_time = end_time - start_time
    print(calculate_z_serial.__name__ + " took", execution_time, "seconds")

    assert sum(result) == 33219980  # Expected sum for a 1000^2 grid with 300 iterations

# Execute the function with given parameters
generate_julia(draw_output=False, desired_width=1000, max_iterations=300)



Length of x values: 1000
Total elements: 1000000
calculate_z_serial took 11.605656385421753 seconds
peak memory: 166.72 MiB, increment: 79.73 MiB
